# Sudoku (linear programming)

Note to self: remember to upload this to github sometime after you set up git pages.

- Document the problem solving process:
  - Realisation of parallels with linear problems
  - Reference to existing LP sudoku solvers (pulp's docs)
  - Extension to accomodate cursed rules
- Highlight your ability to frame problems under a LP framework
- Discuss further applications:
  - 1-9 sudoku, which has more complex logic
- Describe the parallels to arbitrary resource allocation problems
  - Scheduling subcommittee meetings subject to time constraints and venue constraints (rjazz band prac for example)

In [137]:
# linear programming library
!pip install pulp
from pulp import *

In [0]:
NUMS = "123456789"

def make_grid(N = 3):
  nums = NUMS[:N**2]
  prob = LpProblem("sudoku", LpMinimize)
  vals = LpVariable.dicts("vals", (nums, nums, nums), 0, 1, LpInteger)
  
  return prob, vals

def show_vals(vals, N = 3):
  nums = NUMS[:N**2]
  buffer = ((2*N+1)*'-').join((N+1)*['+'])
  
  for r in nums:

    if r in nums[::N]:
      print(buffer)

    for c in nums:

      if c in nums[::N]:
        print('|', end = ' ')

      for n in nums:
        if value(vals[r][c][n]) == 1:
          print(n, end = ' ')

      if c in nums[-1]:
        print('|')

    if r in nums[-1]:
      print(buffer)

##  Regular sudoku

Trial on normal sudoku before we attempt cursed sudoku

In [0]:
def fill_grid(prob, vals, fill, N = 3):  
  nums = NUMS[:N**2]
  
  cells = [(r, c) for r in nums for c in nums]
  fills = [n for n in fill if n in nums + '.0']
  
  for (r, c), n in zip(cells, fills):
    if n in nums:
      prob += vals[r][c][n] == 1, ""
      
  return prob, vals

def add_normal_constraints(prob, vals, N = 3):
  nums = NUMS[:N**2]
  prob += 0, "Arbitrary objective"
  
  for r in nums:
    for c in nums:
      prob += lpSum([vals[r][c][n] for n in nums]) == 1, ""

  for n in nums:
    for r in nums:
      prob += lpSum([vals[r][c][n] for c in nums]) == 1, ""

    for c in nums:
      prob += lpSum([vals[r][c][n] for r in nums]) == 1, ""

    for i in range(N):
      for j in range(N):
        R = nums[N*i:N*(i+1)]
        C = nums[N*j:N*(j+1)]
        prob += lpSum([vals[r][c][n] for r in R for c in C]) == 1, ""
    
  return prob, vals

In [153]:
# set up linear problem

fill = """
530070000
600195000
098000060
800060003
400803001
700020006
060000280
000419005
000080079
"""

sudoku, solution = make_grid()
sudoku, solution = fill_grid(sudoku, solution, fill)
sudoku, solution = add_normal_constraints(sudoku, solution)

# solve linear problem
sudoku.solve()
show_vals(solution)

+-------+-------+-------+
| 5 3 4 | 6 7 8 | 9 1 2 |
| 6 7 2 | 1 9 5 | 3 4 8 |
| 1 9 8 | 3 4 2 | 5 6 7 |
+-------+-------+-------+
| 8 5 9 | 7 6 1 | 4 2 3 |
| 4 2 6 | 8 5 3 | 7 9 1 |
| 7 1 3 | 9 2 4 | 8 5 6 |
+-------+-------+-------+
| 9 6 1 | 5 3 7 | 2 8 4 |
| 2 8 7 | 4 1 9 | 6 3 5 |
| 3 4 5 | 2 8 6 | 1 7 9 |
+-------+-------+-------+


## Cursed sudoku

May this code release my soul from its grasp

In [0]:
# cage constraints
def add_cursed_sums(prob, vals, cursed, N = 3):
  nums = NUMS[:N**2]
  for cells, checksum in cursed.items():
    prob += lpSum([int(n)*vals[r][c][n] for r,c in cells for n in nums]) == checksum, ""
    
  return prob, vals

def add_cursed_ineq(prob, vals, cursed, N = 3):
  nums = NUMS[:N**2]
  for lhs, ineq, rhs in cursed:
    checkval = [] # lhs [op] rhs -> lhs - rhs [op] 0
    checkval += [+int(n)*vals[r][c][n] for r,c in lhs for n in nums]
    checkval += [-int(n)*vals[r][c][n] for r,c in rhs for n in nums]
    
    if ineq in ('==', '='):
      prob += lpSum(checkval) == 0, ""
    elif ineq in ('<'):
      prob += lpSum(checkval) <= 1, ""
    elif ineq in ('>'):
      prob += lpSum(checkval) >= 1, ""
    else:
      raise ValueError('Invalid inequality')
      
  return prob, vals

Toy run on smaller grid

In [198]:
spam = {
    ('11', '22', '33', '44'): 7,
}

eggs = [
    (('12', '21'), '>', ('11', '22')),
]

foo, bar = make_grid(2)
foo, bar = fill_grid(foo, bar, '................', 2)
foo, bar = add_normal_constraints(foo, bar, 2)
foo, bar = add_cursed_sums(foo, bar, spam, 2)
foo, bar = add_cursed_ineq(foo, bar, eggs, 2)

foo.solve()
show_vals(bar, 2)

+-----+-----+
| 1 3 | 4 2 |
| 4 2 | 1 3 |
+-----+-----+
| 2 1 | 3 4 |
| 3 4 | 2 1 |
+-----+-----+


RELEASE MY SOUL

In [202]:
cursed_sums = {
    ('11', '12'): 5,
    ('13', '14'): 7,
    ('16', '17'): 13,
    ('23', '33'): 17,
    ('26', '35', '36'): 9,
    ('32', '42', '52', '62', '72'): 23,
    ('43', '44'): 12,
    ('45', '46', '55', '56'): 23,
    ('53', '63', '73', '74', '75'): 27,
    ('66', '76'): 10,
    ('69', '79'): 9,
    ('77', '78'): 3,
    ('81', '82', '91', '92'): 16,
    ('83', '84', '85', '86', '87'): 30,
    ('88', '98'): 13,
    ('89', '99'): 11,
    ('96', '97'): 6,
}

cursed_ineq = [
    (('13', '14'), '>', ('24', '34')),
    (('21', '22'), '>', ('31', '41')),
    (('47', '48', '57'), '=', ('58', '59')),
    (('51', '61', '71'), '>', ('32', '42', '52', '62', '72')),
    (('67', '68'), '>', ('58', '59')),
]

cursed_sudoku, cursed_solution = make_grid()
add_normal_constraints(cursed_sudoku, cursed_solution)
add_cursed_sums(cursed_sudoku, cursed_solution, cursed_sums)
add_cursed_ineq(cursed_sudoku, cursed_solution, cursed_ineq)

cursed_sudoku.solve()
show_vals(cursed_solution)

+-------+-------+-------+
| 3 2 1 | 6 7 8 | 5 4 9 |
| 6 7 8 | 4 9 5 | 2 3 1 |
| 4 5 9 | 2 1 3 | 6 8 7 |
+-------+-------+-------+
| 5 6 4 | 8 3 9 | 7 1 2 |
| 9 8 2 | 1 4 7 | 3 5 6 |
| 7 1 3 | 5 2 6 | 8 9 4 |
+-------+-------+-------+
| 8 3 7 | 9 6 4 | 1 2 5 |
| 2 4 5 | 7 8 1 | 9 6 3 |
| 1 9 6 | 3 5 2 | 4 7 8 |
+-------+-------+-------+
